<a href="https://colab.research.google.com/github/yunsing/Compsci361/blob/master/LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LOF ANOMALY DETECTION

First we generate the data to be used this is similar to what we seen in class.

In [0]:
from collections import defaultdict
import numpy as np

instances = np.matrix([[0,0],[0,1],[1,1],[3,0]])

import numpy as np
import matplotlib.pyplot as plt

x = np.squeeze(np.asarray(instances[:,0]))
y = np.squeeze(np.asarray(instances[:,1]))
plt.cla()
plt.figure(1)
plt.scatter(x,y)
plt.show()


Use  a Manhattan distance to caluclate distance.

In [0]:
k = 2
distance = 'manhattan'


from sklearn.metrics import pairwise_distances
dist = pairwise_distances(instances,metric=distance)

Actually calculating k- distance

In [0]:
# Calculate K distance
import heapq
k_distance = defaultdict(tuple)
# For each data point
for i in range(instances.shape[0]):
    # Get its distance to all the other points.
    # Convert array into list for convienience
    distances = dist[i].tolist()
    # Get the K nearest neighbours
    ksmallest = heapq.nsmallest(k+1,distances)[1:][k-1]
    # Get their indices
    ksmallest_idx = distances.index(ksmallest)
    # For each data point store the K th nearest neighbour and its distance
    k_distance[i]=(ksmallest,ksmallest_idx)

Calculate K distance neighbourhood

In [0]:
def all_indices(value, inlist):
    out_indices = []
    idx = -1
    while True:
        try:
            idx = inlist.index(value, idx+1)
            out_indices.append(idx)
        except ValueError:
            break
    return out_indices
# Calculate K distance neighbourhood
import heapq
k_distance_neig = defaultdict(list)
# For each data point
for i in range(instances.shape[0]):
    # Get the points distances to its neighbours
    distances = dist[i].tolist()
    print ("k distance neighbourhood",i)
    print (distances)
    # Get the 1 to K nearest neighbours
    ksmallest = heapq.nsmallest(k+1,distances)[1:]
    print (ksmallest)
    ksmallest_set = set(ksmallest)
    print (ksmallest_set)
    ksmallest_idx = []
    # Get the indices of the K smallest elements
    for x in ksmallest_set:
            ksmallest_idx.append(all_indices(x,distances))
    # Change a list of list to list
    ksmallest_idx = [item for sublist in ksmallest_idx for item in sublist]
    # For each data pont store the K distance neighbourhood
    k_distance_neig[i].extend(zip(ksmallest,ksmallest_idx))



Calculate LRD (Local reachable density)

In [0]:
#Local reachable density
local_reach_density = defaultdict(float)
for i in range(instances.shape[0]):
    # LRDs numerator, number of K distance neighbourhood
    no_neighbours = len(k_distance_neig[i])
    denom_sum = 0
    # Reachability distance sum
    for neigh in k_distance_neig[i]:
        # maximum(K-Distance(P), Distance(P,Q))
        denom_sum+=max(k_distance[neigh[1]][0],neigh[0])
    local_reach_density[i] = no_neighbours/(1.0*denom_sum)
    print("Local Reach Density:", i, ",", local_reach_density[i])

Calculate K distance neighbourhood

In [0]:
lof_list =[]
#Local Outlier Factor
for i in range(instances.shape[0]):
    lrd_sum = 0
    rdist_sum = 0
    for neigh in k_distance_neig[i]:
        lrd_sum+=local_reach_density[neigh[1]]
        rdist_sum+=max(k_distance[neigh[1]][0],neigh[0])
    lof_list.append((i,lrd_sum*rdist_sum))
print("LOF list:",lof_list)